In [1]:
import numpy as np
import pennylane as qml
from pennylane import qchem as qchem
from pennylane import numpy as np

In [2]:
#load the molecule with nuclear coordinates
symbols = ['H', 'N', 'H', 'H']
coordinates = np.array([0.0, 0.0, 0.0, 0.9989, 0.0000, 0.0000, 1.3259, 0.9425, 0.0000, 1.3211, -0.4565, -0.8272])

In [3]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print('Number of qubits:', qubits)
print('Hamiltonian:', H)

/opt/anaconda3/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


Number of qubits: 16
Hamiltonian:   (-27.286700163373574) [I0]
+ (0.13135984600247191) [Z10]
+ (0.13135984600247197) [Z11]
+ (0.22157616008252196) [Z15]
+ (0.22157616008252218) [Z14]
+ (0.22275126791943273) [Z12]
+ (0.2227512679194329) [Z13]
+ (1.0394002161304712) [Z9]
+ (1.0394002161304714) [Z8]
+ (1.1613993601373154) [Z7]
+ (1.161399360137316) [Z6]
+ (1.1621545570063663) [Z4]
+ (1.1621545570063667) [Z5]
+ (1.439985683149263) [Z2]
+ (1.4399856831492632) [Z3]
+ (9.50190647057729) [Z0]
+ (9.50190647057729) [Z1]
+ (-0.009907967494522623) [Y8 Y10]
+ (-0.009907967494522623) [X8 X10]
+ (-0.005598297341544352) [Y3 Y5]
+ (-0.005598297341544352) [X3 X5]
+ (-0.0004323337853491323) [Y6 Y8]
+ (-0.0004323337853491323) [X6 X8]
+ (-3.724925809461734e-05) [Y12 Y14]
+ (-3.724925809461734e-05) [X12 X14]
+ (-8.441885081333401e-06) [Y5 Y7]
+ (-8.441885081333401e-06) [X5 X7]
+ (5.683067649909362e-06) [Y13 Y15]
+ (5.683067649909362e-06) [X13 X15]
+ (6.470720951504399e-06) [Y7 Y9]
+ (6.470720951504399e-06) 

In [4]:
dev = qml.device("default.qubit", wires=qubits)

In [5]:
electrons = 2
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

In [7]:
@qml.qnode(dev)
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

In [8]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [9]:
theta = np.array(0.0, requires_grad=True)

In [ ]:
# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(12)

# Full configuration interaction (FCI) energy computed classically
E_fci = -1.136189454088

# Add energy plot on column 1
ax1 = fig.add_subplot(121)
ax1.plot(range(n + 2), energy, "go", ls="dashed")
ax1.plot(range(n + 2), np.full(n + 2, E_fci), color="red")
ax1.set_xlabel("Optimization step", fontsize=13)
ax1.set_ylabel("Energy (Hartree)", fontsize=13)
ax1.text(0.5, -1.1176, r"$E_\mathrm{HF}$", fontsize=15)
ax1.text(0, -1.1357, r"$E_\mathrm{FCI}$", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add angle plot on column 2
ax2 = fig.add_subplot(122)
ax2.plot(range(n + 2), angle, "go", ls="dashed")
ax2.set_xlabel("Optimization step", fontsize=13)
ax2.set_ylabel("Gate parameter $\\theta$ (rad)", fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.subplots_adjust(wspace=0.3, bottom=0.2)
plt.show()